![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Use pretrained `explain_document` Pipeline

In [19]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.5.1

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)
Processing /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471/pyspark-2.4.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e3/53/c737818eb9a7dc32a7cd4f1396e787bd94200c3997c72c1dbe028587bd76/py4j-0.10.7-py2.py3-none-any.whl


  Using cached https://files.pythonhosted.org/packages/df/b4/db653f8080a446de8ce981b262d85c85c61de7e920930726da0d1c6b4c65/spark_nlp-2.5.1-py2.py3-none-any.whl


### Stages

 * DocumentAssembler
 * SentenceDetector
 * Tokenizer
 * Lemmatizer
 * Stemmer
 * Part of Speech
 * SpellChecker (Norvig)

In [ ]:
import sys
import time

#Spark ML and SQL
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.functions import array_contains
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
#Spark NLP
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import DocumentAssembler, Finisher

### Let's create a Spark Session for our app

In [ ]:
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.5.1
Apache Spark version:  2.4.4


#### This is our testing document, we'll use it to exemplify all different pipeline stages.

In [ ]:
testDoc = [
"French author who helped pioner the science-fiction genre. \
Verne wrate about space, air, and underwater travel before \
navigable aircrast and practical submarines were invented, \
and before any means of space travel had been devised. "
]

In [ ]:
pipeline = PretrainedPipeline('explain_document_ml', lang='en')

explain_document_ml download started this may take some time.
Approx size to download 9.4 MB
[OK!]


#### We are not interested in handling big datasets, let's switch to LightPipelines for speed.

In [ ]:
result = pipeline.annotate(testDoc)

#### Let's analyze these results - first let's see what sentences we detected

In [ ]:
[content['sentence'] for content in result]

[['French author who helped pioner the science-fiction genre.',
  'Verne wrate about space, air, and underwater travel before navigable aircrast and practical submarines were invented, and before any means of space travel had been devised.']]

#### Now let's see how those sentences were tokenized

In [ ]:
[content['token'] for content in result] 

[['French',
  'author',
  'who',
  'helped',
  'pioner',
  'the',
  'science-fiction',
  'genre',
  '.',
  'Verne',
  'wrate',
  'about',
  'space',
  ',',
  'air',
  ',',
  'and',
  'underwater',
  'travel',
  'before',
  'navigable',
  'aircrast',
  'and',
  'practical',
  'submarines',
  'were',
  'invented',
  ',',
  'and',
  'before',
  'any',
  'means',
  'of',
  'space',
  'travel',
  'had',
  'been',
  'devised',
  '.']]

#### Notice some spelling errors? the pipeline takes care of that as well

In [ ]:
[content['spell'] for content in result]

[['French',
  'author',
  'who',
  'helped',
  'pioneer',
  'the',
  'sciencefiction',
  'genre',
  '.',
  'Verne',
  'wrote',
  'about',
  'space',
  ',',
  'air',
  ',',
  'and',
  'underwater',
  'travel',
  'before',
  'navigable',
  'aircraft',
  'and',
  'practical',
  'submarines',
  'were',
  'invented',
  ',',
  'and',
  'before',
  'any',
  'means',
  'of',
  'space',
  'travel',
  'had',
  'been',
  'devised',
  '.']]

#### Now let's see the lemmas

In [ ]:
[content['lemmas'] for content in result]

[['French',
  'author',
  'who',
  'help',
  'pioneer',
  'the',
  'sciencefiction',
  'genre',
  '.',
  'Verne',
  'write',
  'about',
  'space',
  ',',
  'air',
  ',',
  'and',
  'underwater',
  'travel',
  'before',
  'navigable',
  'aircraft',
  'and',
  'practical',
  'submarine',
  'be',
  'invent',
  ',',
  'and',
  'before',
  'any',
  'mean',
  'of',
  'space',
  'travel',
  'have',
  'be',
  'devise',
  '.']]

#### Let's check the stems, any difference with the lemmas shown bebore?

[content['lemmas'] for content in result]

In [ ]:
[content['stems'] for content in result]

[['french',
  'author',
  'who',
  'help',
  'pioneer',
  'the',
  'sciencefict',
  'genr',
  '.',
  'vern',
  'wrote',
  'about',
  'space',
  ',',
  'air',
  ',',
  'and',
  'underwat',
  'travel',
  'befor',
  'navig',
  'aircraft',
  'and',
  'practic',
  'submarin',
  'were',
  'invent',
  ',',
  'and',
  'befor',
  'ani',
  'mean',
  'of',
  'space',
  'travel',
  'had',
  'been',
  'devis',
  '.']]

#### Now it's the turn on Part Of Speech(POS)

In [ ]:
pos = [content['pos'] for content in result]
token = [content['token'] for content in result]
# let's put token and tag together
list(zip(token[0], pos[0]))

[('French', 'JJ'),
 ('author', 'NN'),
 ('who', 'WP'),
 ('helped', 'VBD'),
 ('pioner', 'NN'),
 ('the', 'DT'),
 ('science-fiction', 'NN'),
 ('genre', 'NN'),
 ('.', '.'),
 ('Verne', 'NNP'),
 ('wrate', 'VBD'),
 ('about', 'IN'),
 ('space', 'NN'),
 (',', ','),
 ('air', 'NN'),
 (',', ','),
 ('and', 'CC'),
 ('underwater', 'JJ'),
 ('travel', 'NN'),
 ('before', 'IN'),
 ('navigable', 'JJ'),
 ('aircrast', 'NN'),
 ('and', 'CC'),
 ('practical', 'JJ'),
 ('submarines', 'NNS'),
 ('were', 'VBD'),
 ('invented', 'VBN'),
 (',', ','),
 ('and', 'CC'),
 ('before', 'IN'),
 ('any', 'DT'),
 ('means', 'NNS'),
 ('of', 'IN'),
 ('space', 'NN'),
 ('travel', 'NN'),
 ('had', 'VBD'),
 ('been', 'VBN'),
 ('devised', 'VBN'),
 ('.', '.')]

# Use pretrained `match_chunk` Pipeline for Individual Noun Phrase 

* DocumentAssembler
* SentenceDetector
* Tokenizer
* Part of speech
* chunker

Pipeline:
* The pipeline uses regex `<DT>?<JJ>*<NN>+`
* which states that whenever the chunk finds an optional determiner 
* (DT) followed by any number of adjectives (JJ) and then a noun (NN) then the Noun Phrase(NP) chunk should be formed.

In [ ]:
pipeline = PretrainedPipeline('match_chunks', lang='en')

match_chunks download started this may take some time.
Approx size to download 4.3 MB
[OK!]


In [ ]:
result = pipeline.annotate("The book has many chapters") # single noun phrase

In [ ]:
result['chunk']

['The book']

In [ ]:
result = pipeline.annotate("the little yellow dog barked at the cat") #multiple noune phrases

In [ ]:
result['chunk']

['the little yellow dog', 'the cat']